In [1]:
import httpx
import pandas as pd
import sqlmodel

from arpav_ppcv import (
    config,
    database,
    operations,
)
from arpav_ppcv.schemas.base import ObservationDataSmoothingStrategy

settings = config.get_settings()
session = sqlmodel.Session(database.get_engine(settings))

In [2]:
variable = database.get_variable_by_name(session, "TDd")
station = database.get_station_by_code(session, "93")

In [3]:
df = operations.get_observation_time_series(
    settings,
    session,
    variable,
    station,
    month=1,
    temporal_range="../..",
    smoothing_strategy=ObservationDataSmoothingStrategy.MOVING_AVERAGE_5_YEARS,
    include_decade_data=True
)

In [4]:
df.info()

In [5]:
df

In [8]:
measurements = database.collect_all_monthly_measurements(
    session,
    station_id_filter=station.id,
    variable_id_filter=variable.id,
    month_filter=1,
)

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame([m.model_dump() for m in measurements])

In [11]:
df = df[["value", "date"]].rename(columns={"value": variable.name})

In [12]:
df["time"] = pd.to_datetime(df["date"], utc=True)

In [13]:
df = df[[variable.name, "time"]]

In [14]:
df.set_index("time", inplace=True)

In [15]:
df

In [16]:
decade_df = df.groupby((df.index.year//10)*10).mean()

In [27]:
df["decade"] = (df.index.year // 10) * 10

In [36]:
merged = df.merge(decade_df, how="inner", left_on="decade", right_on="time")

In [37]:
merged

In [42]:
decade_df

In [50]:
df.join(decade_df, on="decade", rsuffix="_decade")